In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
import wave
import pyroomacoustics as pa
import glob
import random

from tqdm import tqdm

# 乱数を初期化
random.seed(0)

In [2]:
# 音声データをロードし、指定された秒数とサンプリングレートでリサンプル
def load_audio_file(file_path, length, sample_rate):
    data, sr = sf.read(file_path)
    # データが設定値よりも大きい場合は大きさを超えた分をカットする
    # データが設定値よりも小さい場合はデータの後ろを0でパディングする
    # シングルチャンネル(モノラル)の場合 (data.shape: [num_samples,])
    if data.ndim == 1:
        if len(data) > sample_rate*length:
            data = data[:sample_rate*length]
        else:
            data = np.pad(data, (0, max(0, sample_rate*length - len(data))), "constant")
        """data: (num_samples, )"""
    # マルチチャンネルの場合 (data.shape: [num_samples, num_channels])
    elif data.ndim == 2:
        if data.shape[0] > sample_rate*length:
            data = data[:sample_rate*length, :]
        else:
            data = np.pad(data, [(0, max(0, sample_rate*length-data.shape[0])), (0, 0)], "constant")
        """data: (num_samples, num_channels)"""
    else:
        print("number of audio channels are incorrect")
    return data

In [3]:
# 音声データを指定したサンプリングレートで保存
def save_audio_file(file_path, data, sample_rate):
    # librosa.output.write_wav(file_path, data, sample_rate) # 正常に動作しないので変更
    sf.write(file_path, data, sample_rate)

In [4]:
# 2つのオーディオデータを足し合わせる
def audio_mixer(data1, data2):
    assert len(data1) == len(data2)
    mixed_audio = data1 + data2
    return mixed_audio

In [5]:
# 音声データをスペクトログラムに変換する
def wave_to_spec(data, fft_size, hop_length):
    # 短時間フーリエ変換(STFT)を行い、スペクトログラムを取得
    spec = librosa.stft(data, n_fft=fft_size, hop_length=hop_length)
    amp = np.abs(spec) # 振幅スペクトログラムを取得
    phase = np.exp(1.j * np.angle(spec)) # 位相スペクトログラムを取得(フェーザ表示)
    # mel_spec = librosa.feature.melspectrogram(data, sr=sr, n_mels=128) # メルスペクトログラムを用いる場合はこっちを使う
    return amp, phase

In [6]:
# 音声に室内インパルス応答（Room Impulse Response）を畳み込む
def rir_convolve(wave_files, sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR):
    """
    wave_files: シングルチャンネルの音声のパスを格納したリスト
    sample_rate: サンプリング周波数 [Hz]
    audio_length: 音声の長さ [sec]
    doas: 音源の到来方向
    distance_mic_to_source: 音源とマイクロホンの距離 [m]
    mic_array_loc: マイクロホンアレイの位置座標
    R: 各マイクロホンの空間的な座標
    room_dim: 部屋の３次元形状を表す（単位はm）
    max_order: 部屋の壁で何回音が反射するか（反射しない場合0）
    absorption: 部屋の壁でどの程度音が吸収されるか （吸収されない場合None）
    SNR: 音声と雑音の比率 [dB]
    """
    n_sources = len(wave_files)
#     print("音源数:", n_sources)
    source_locations = np.zeros((3, doas.shape[0]), dtype=doas.dtype)
    """source_locations: (xyz, num_sources)"""
    source_locations[0,  :] = np.cos(doas[:, 1]) * np.cos(doas[:, 0]) # x = rcosφcosθ
    source_locations[1,  :] = np.sin(doas[:, 1]) * np.cos(doas[:, 0]) # y = rsinφcosθ
    source_locations[2,  :] = np.sin(doas[:, 0]) # z = rsinθ
    source_locations *= distance_mic_to_source
    source_locations += mic_array_loc[:, None] # マイクロホンアレイからの相対位置→絶対位置
    for i in range(n_sources):
        x = source_locations[0, i]
        y = source_locations[1, i]
        z = source_locations[2, i]
#         print("{}個目の音源の位置： (x, y, z) = ({}, {}, {})".format(i+1, x, y, z))

    # 音源数分の音声ファイルを読み込む
    clean_data = np.zeros([n_sources, sample_rate*audio_length], dtype='float64')
    for s, wave_file in enumerate(wave_files):
        audio_data = load_audio_file(wave_file, audio_length, sample_rate)
        clean_data[s, :] = audio_data
        
    # 部屋を生成する
    room = pa.ShoeBox(room_dim, fs=sample_rate, max_order=max_order, absorption=absorption)
    # 用いるマイクロホンアレイの情報を設定する
    room.add_microphone_array(pa.MicrophoneArray(R, fs=room.fs))
    # 各音源をシミュレーションに追加する
    for s in range(n_sources):
        clean_data[s] /= np.std(clean_data[s])
        # たまに「ValueError: The source must be added inside the room.」が出る
        room.add_source(source_locations[:, s], signal=clean_data[s])
    # RIRのシミュレーション生成と音源信号への畳み込みを実行
    room.simulate(snr=SNR)
#     # インパルス応答の取得と残響時間（RT60）の取得
#     impulse_responses = room.rir
#     rt60 = pa.experimental.measure_rt60(impulse_responses[0][0], fs=sampling_rate)
#     print("残響時間:{} [sec]".format(rt60))
    # 室内インパルス応答を畳み込んだ波形データを取得
    convolved_wave = room.mic_array.signals.T
    """convolved_wave: (num_samples, num_channels)"""
    
    return convolved_wave

In [7]:
if __name__ == '__main__':
    # 各パラメータを設定
    sample_rate = 16000 # 作成するオーディオファイルのサンプリング周波数を指定
    audio_length = 3 # 単位は秒(second) → fft_size=1024,hop_length=768のとき、audio_length=6が最適化かも？
    train_val_ratio = 0.9 # trainデータとvalidationデータの割合
    fft_size = 512 # 短時間フーリエ変換のフレーム長
    hop_length = 160 # 短時間フーリエ変換においてフレームをスライドさせる幅
    gain_decay = 0.8 # 音量調整のためのパラメータ（雑音が大きすぎるため）
    
    # RIR生成用のパラメータ
    # 畳み込みに用いる波形
    # clean_wave_files = ["../data/NoisySpeechDatabase/noisy_trainset_28spk_wav_16kHz/p230_013.wav"]
    # 音声と雑音の比率 [dB]
    SNR = None
    # 音源とマイクロホンの距離 [m]
    distance_mic_to_source=2
#     # 音源方向（音源が複数ある場合はリストに追加）
#     azimuth = [0] # 方位角
#     elevation = [np.pi/6] # 仰角
    # 部屋（シミュレーション環境）の設定
    room_width = 5.0
    room_length = 5.0
    room_height = 5.0
    # 部屋の残響を設定
    max_order = 0 # 部屋の壁で何回音が反射するか（反射しない場合0）
    absorption = None # 部屋の壁でどの程度音が吸収されるか （吸収されない場合None）
    # 以下は固定
    # 部屋の３次元形状を表す（単位はm）
    room_dim = np.r_[room_width, room_length, room_height]
    print("部屋の3次元形状：", room_dim)
    # マイクロホンアレイの中心位置
    nakbot_height = 0.57 # Nakbotの全長
    mic_array_height = nakbot_height - 0.04 # 0.04はTAMAGO-03マイクロホンアレイの頂上部からマイクロホンアレイ中心までの距離
    mic_array_loc = np.r_[room_width/2, room_length/2, 0] + [0, 0, mic_array_height] # 部屋の中央に配置されたNakbot上のマイクロホンアレイ
    print("マイクロホンアレイ中心座標：", mic_array_loc)
    # TAMAGO-03のマイクロホンアレイのマイクロホン配置（単位はm）
    mic_alignments = np.array(
    [
        [0.035, 0.0, 0.0],
        [0.035/np.sqrt(2), 0.035/np.sqrt(2), 0.0],
        [0.0, 0.035, 0.0],
        [-0.035/np.sqrt(2), 0.035/np.sqrt(2), 0.0],
        [-0.035, 0.0, 0.0],
        [-0.035/np.sqrt(2), -0.035/np.sqrt(2), 0.0],
        [0.0, -0.035, 0.0],
        [0.035/np.sqrt(2), -0.035/np.sqrt(2), 0.0]
    ])
    n_channels = np.shape(mic_alignments)[0]
    print("マイクロホン数：", n_channels)
    # get the microphone array （各マイクロホンの空間的な座標）
    R = mic_alignments.T + mic_array_loc[:, None]
    """R: (3D coordinates [m], num_microphones)"""
#     # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
#     doas = np.array(
#     [[elevation[0], azimuth[0]], # １個目の音源 
#     # [elevation[1], azimuth[1]] # ２個目の音源
#     ])
    
    # データセットを格納するディレクトリを作成
    save_dataset_dir = "../data/NoisySpeechDataset_for_unet_fft_512_multi_wav_1207/"
    os.makedirs(save_dataset_dir, exist_ok=True)

    # 学習・評価用
    # 人の発話音声のディレクトリを指定
    target_data_dir = "../data/NoisySpeechDatabase/clean_trainset_28spk_wav_16kHz/"
    # 外部雑音のディレクトリを指定
    interference_data_dir = "../data/NoisySpeechDatabase/interference_trainset_wav_16kHz/"
    # 混合音声のディレクトリを指定
    mixed_data_dir = "../data/NoisySpeechDatabase/noisy_trainset_28spk_wav_16kHz/"

    target_data_path_template = os.path.join(target_data_dir, "*.wav")
    target_list = glob.glob(target_data_path_template)
    # データセットをシャッフル
    random.shuffle(target_list)
    # データをtrainデータとvalidationデータに分割
    target_list_for_train = target_list[:int(len(target_list)*train_val_ratio)]
    target_list_for_train = random.sample(target_list_for_train, 3500) # データ量削減（7の倍数に指定）
    target_list_for_val = target_list[int(len(target_list)*train_val_ratio):]
    target_list_for_val = random.sample(target_list_for_val, 350) # データ量削減（7の倍数に指定）
    print("オリジナルデータの数:", len(target_list))
    print("trainデータの数:", len(target_list_for_train))
    print("validationデータの数:", len(target_list_for_val))
    
    # テスト用
    # 人の発話音声のディレクトリを指定
    target_data_dir_for_test= "../data/NoisySpeechDatabase/clean_testset_wav_16kHz/"
    # 外部雑音のディレクトリを指定
    interference_data_dir_for_test = "../data/NoisySpeechDatabase/interference_testset_wav_16kHz/"
    # 混合音声のディレクトリを指定
    mixed_data_dir_for_test = "../data/NoisySpeechDatabase/noisy_testset_wav_16kHz/"
    # テストデータのリストを作成
    target_data_path_template_for_test = os.path.join(target_data_dir_for_test, "*.wav")
    target_list_for_test = glob.glob(target_data_path_template_for_test)
    target_list_for_test = random.sample(target_list_for_test, 105) # データ量削減（7の倍数に指定）
    print("testデータの数:", len(target_list_for_test))
    
    # trainデータを作成
    print("trainデータ作成中")
    train_data_path = os.path.join(save_dataset_dir, "train")
    os.makedirs(train_data_path, exist_ok=True)
    for idx, target_path in enumerate(tqdm(target_list_for_train)):
        file_num = os.path.basename(target_path).split('.')[0] # (例)p226_001
        target_file_name = file_num + "_target.wav" # (例)p226_001_target.wav
        
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        # 目的音の畳み込み
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [0] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
#         [elevation[1], azimuth[1]] # ２個目の音源
        ])
        convolved_target_data = rir_convolve([target_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_target_data = convolved_target_data[:sample_rate*audio_length, :]
        """convolved_target_data: (num_samples, num_channels=8)"""
        
        # 干渉音の畳み込み
        interference_path = os.path.join(interference_data_dir, file_num + ".wav")
        # 干渉音の到来方向を指定（0°, 15°, 30°, 45°, 60°, 75°, 90°の7分割）
        interference_azimuth = int(idx / (len(target_list_for_train) / 7)) * (np.pi / 12)
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [interference_azimuth] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
#         [elevation[1], azimuth[1]] # ２個目の音源
        ])
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_interference_data = rir_convolve([interference_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_interference_data = convolved_interference_data[:sample_rate*audio_length, :]
        """convolved_interference_data: (num_samples, num_channels=8)"""
#         convolved_interference_data = convolved_interference_data * gain_decay
        
        # 畳み込む音声をリストに格納
        wave_files = [target_path, interference_path]
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [0, interference_azimuth] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6, np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
        [elevation[1], azimuth[1]] # ２個目の音源
        ])
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_mixed_data = rir_convolve(wave_files, sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分音声データが長くなるので、元に戻す
        convolved_mixed_data = convolved_mixed_data[:sample_rate*audio_length, :]
        """convolved_mixed_data: (num_samples, num_channels=8)"""
    
        # 混合音声の最大振幅で正規化
        normalized_convolved_target_data = convolved_target_data / convolved_mixed_data.max()
        normalized_convolved_interference_data = convolved_interference_data / convolved_mixed_data.max()
        normalized_convolved_mixed_data = convolved_mixed_data / convolved_mixed_data.max()
        # 音声を保存
        # 目的音
        target_file_path = os.path.join(train_data_path, target_file_name)
        save_audio_file(target_file_path, normalized_convolved_target_data, sample_rate)
        # 干渉音
        interference_file_name = file_num + "_interference.wav" # (例)p226_001_interference.wav
        interference_file_path = os.path.join(train_data_path, interference_file_name)
        save_audio_file(interference_file_path, normalized_convolved_interference_data, sample_rate)
        # 混合音声
        mixed_file_name = file_num + "_mixed.wav" # (例)p226_001_mixed.wav
        mixed_file_path = os.path.join(train_data_path, mixed_file_name)
        save_audio_file(mixed_file_path, normalized_convolved_mixed_data, sample_rate)
        
        
    # validationデータを作成
    print("validationデータ作成中")
    val_data_path = os.path.join(save_dataset_dir, "val")
    os.makedirs(val_data_path, exist_ok=True)
    for idx, target_path in enumerate(tqdm(target_list_for_val)):
        file_num = os.path.basename(target_path).split('.')[0] # (例)p226_001
        target_file_name = file_num + "_target.wav" # (例)p226_001_target.wav
        
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        # 目的音の畳み込み
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [0] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
#         [elevation[1], azimuth[1]] # ２個目の音源
        ])
        convolved_target_data = rir_convolve([target_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_target_data = convolved_target_data[:sample_rate*audio_length, :]
        
        # 干渉音の畳み込み
        interference_path = os.path.join(interference_data_dir, file_num + ".wav")
        # 干渉音の到来方向を指定（0°, 15°, 30°, 45°, 60°, 75°, 90°の7分割）
        interference_azimuth = int(idx / (len(target_list_for_val) / 7)) * (np.pi / 12)
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [interference_azimuth] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
#         [elevation[1], azimuth[1]] # ２個目の音源
        ])
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_interference_data = rir_convolve([interference_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_interference_data = convolved_interference_data[:sample_rate*audio_length, :]
        """convolved_interference_data: (num_samples, num_channels=8)"""
#         convolved_interference_data = convolved_interference_data * gain_decay

        # 畳み込む音声をリストに格納
        wave_files = [target_path, interference_path]
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [0, interference_azimuth] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6, np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
        [elevation[1], azimuth[1]] # ２個目の音源
        ])
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_mixed_data = rir_convolve(wave_files, sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分音声データが長くなるので、元に戻す
        convolved_mixed_data = convolved_mixed_data[:sample_rate*audio_length, :]
        """convolved_mixed_data: (num_samples, num_channels=8)"""
        
        # 混合音声の最大振幅で正規化
        normalized_convolved_target_data = convolved_target_data / convolved_mixed_data.max()
        normalized_convolved_interference_data = convolved_interference_data / convolved_mixed_data.max()
        normalized_convolved_mixed_data = convolved_mixed_data / convolved_mixed_data.max()
        # 音声を保存
        # 目的音
        target_file_path = os.path.join(val_data_path, target_file_name)
        save_audio_file(target_file_path, normalized_convolved_target_data, sample_rate)
        # 干渉音
        interference_file_name = file_num + "_interference.wav" # (例)p226_001_interference.wav
        interference_file_path = os.path.join(val_data_path, interference_file_name)
        save_audio_file(interference_file_path, normalized_convolved_interference_data, sample_rate)
        # 混合音声
        mixed_file_name = file_num + "_mixed.wav" # (例)p226_001_mixed.wav
        mixed_file_path = os.path.join(val_data_path, mixed_file_name)
        save_audio_file(mixed_file_path, normalized_convolved_mixed_data, sample_rate)
        
        
    # testデータを作成
    print("testデータ作成中")
    test_data_path = os.path.join(save_dataset_dir, "test")
    os.makedirs(test_data_path, exist_ok=True)
    for idx, target_path in enumerate(tqdm(target_list_for_test)):
        file_num = os.path.basename(target_path).split('.')[0] # (例)p226_001
        target_file_name = file_num + "_target.wav" # (例)p226_001_target.wav
        
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        # 目的音の畳み込み
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [0] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
#         [elevation[1], azimuth[1]] # ２個目の音源
        ])
        convolved_target_data = rir_convolve([target_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_target_data = convolved_target_data[:sample_rate*audio_length, :]
        """convolved_target_data: (num_samples, num_channels=8)"""
        
        # 干渉音の畳み込み
        interference_path = os.path.join(interference_data_dir_for_test, file_num + ".wav")
        # 干渉音の到来方向を指定（0°, 15°, 30°, 45°, 60°, 75°, 90°の7分割）
        interference_azimuth = int(idx / (len(target_list_for_test) / 7)) * (np.pi / 12)
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [interference_azimuth] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
#         [elevation[1], azimuth[1]] # ２個目の音源
        ])
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_interference_data = rir_convolve([interference_path], sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_interference_data = convolved_interference_data[:sample_rate*audio_length, :]
        """convolved_interference_data: (num_samples, num_channels=8)"""
#         convolved_interference_data = convolved_interference_data * gain_decay

        # 畳み込む音声をリストに格納
        wave_files = [target_path, interference_path]
        # 音源方向（音源が複数ある場合はリストに追加、目的音の音源方向は固定）
        azimuth = [0, interference_azimuth] # 方位角（1個目の音源, 2個目の音源）
        elevation = [np.pi/6, np.pi/6] # 仰角（1個目の音源, 2個目の音源）
        # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
        doas = np.array(
        [[elevation[0], azimuth[0]], # １個目の音源 
        [elevation[1], azimuth[1]] # ２個目の音源
        ])
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_mixed_data = rir_convolve(wave_files, sample_rate, audio_length, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分音声データが長くなるので、元に戻す
        convolved_mixed_data = convolved_mixed_data[:sample_rate*audio_length, :]
        """convolved_mixed_data: (num_samples, num_channels=8)"""
    
        # 混合音声の最大振幅で正規化
        normalized_convolved_target_data = convolved_target_data / convolved_mixed_data.max()
        normalized_convolved_interference_data = convolved_interference_data / convolved_mixed_data.max()
        normalized_convolved_mixed_data = convolved_mixed_data / convolved_mixed_data.max()
        # 音声データのサンプリング周波数を指定して保存
        # 目的音
        target_file_path = os.path.join(test_data_path, target_file_name)
        save_audio_file(target_file_path, normalized_convolved_target_data, sample_rate)
        # 干渉音
        interference_azimuth_degree = str(int(idx / (len(target_list_for_test) / 7)) * 15)
        interference_file_name = file_num + "_interference_azimuth{}.wav".format(interference_azimuth_degree) # (例)p226_001_interference_azimuth30.wav
        interference_file_path = os.path.join(test_data_path, interference_file_name)
        save_audio_file(interference_file_path, normalized_convolved_interference_data, sample_rate)
        # 混合音声
        mixed_file_name = file_num + "_mixed_azimuth{}.wav".format(interference_azimuth_degree) # (例)p226_001_mixed_azimuth30.wav
        mixed_file_path = os.path.join(test_data_path, mixed_file_name)
        save_audio_file(mixed_file_path, normalized_convolved_mixed_data, sample_rate)

    print("データ作成完了　保存先：{}".format(save_dataset_dir))

  0%|          | 0/3500 [00:00<?, ?it/s]

部屋の3次元形状： [5. 5. 5.]
マイクロホンアレイ中心座標： [2.5  2.5  0.53]
マイクロホン数： 8
オリジナルデータの数: 11572
trainデータの数: 3500
validationデータの数: 350
testデータの数: 105
trainデータ作成中


  0%|          | 1/350 [00:00<00:37,  9.30it/s]

validationデータ作成中


  1%|          | 1/105 [00:00<00:10,  9.53it/s]

testデータ作成中


100%|██████████| 105/105 [00:14<00:00,  7.37it/s]

データ作成完了　保存先：../data/NoisySpeechDataset_for_unet_fft_512_multi_wav_1207/
